## Паттерны проектирования и их особенности в Python

### Классификация паттернов

**Паттерны проектирования** — это проверенные временем архитектурные решения, которые помогают организовать код и сделать его более понятным, гибким и поддерживаемым. Они формируют "язык" проектирования, позволяя разработчикам быстрее понимать логику и структуру приложений.

Python, благодаря своей динамической природе, предоставляет уникальные способы реализации паттернов, упрощая многие из них. Однако у этого есть и обратная сторона: неправильное применение может привести к избыточной сложности.

Паттерны делятся на три основные группы:

1. **Порождающие**:
   - Решают задачи создания объектов.
   - Примеры: Singleton, Factory, Builder.

2. **Структурные**:
   - Организуют отношения между объектами.
   - Примеры: Adapter, Decorator, Proxy.

3. **Поведенческие**:
   - Описывают взаимодействие между объектами.
   - Примеры: Observer, Strategy, Command.

[Список паттернов на википедии](https://ru.wikipedia.org/wiki/Design_Patterns)

### 1. Singleton

#### Назначение:
Обеспечивает создание только одного экземпляра класса и предоставляет глобальную точку доступа к нему.

#### Реализация в Python через метакласс

```python
class SingletonMeta(type):
    _instances = {}

    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super().__call__(*args, **kwargs)
        return cls._instances[cls]

class SingletonClass(metaclass=SingletonMeta):
    pass

a = SingletonClass()
b = SingletonClass()

print(a is b)  # True
```


#### Когда использовать:

1. Для объектов, которые должны существовать в единственном экземпляре (например, подключение к базе данных, логгирование).
2. Когда нужно обеспечить контроль над созданием экземпляра.

#### Проблемы:
1. Усложняет тестирование (необходима замена экземпляра в тестах).
2. Может нарушить принцип единственной ответственности (SRP), если Singleton берёт на себя слишком много задач.

### 2. Factory (Фабрика)

#### Назначение:
Создаёт объекты без указания точного класса.

#### Пример в Python

```python
class Shape:
    def draw(self):
        pass

class Circle(Shape):
    def draw(self):
        print("Drawing a Circle")

class Square(Shape):
    def draw(self):
        print("Drawing a Square")

class ShapeFactory:
    @staticmethod
    def create_shape(shape_type):
        if shape_type == "circle":
            return Circle()
        elif shape_type == "square":
            return Square()
        else:
            raise ValueError("Unknown shape type")

# Использование
shape = ShapeFactory.create_shape("circle")
shape.draw()  # Drawing a Circle
```

#### Когда использовать:

1. Когда логика создания объектов сложная.
2. Когда точный класс создаваемого объекта неизвестен до выполнения программы.

#### Особенности в Python:
- Вместо фабрики часто используют **функции**, так как они проще и читаемее.


### 3. Builder (Строитель)

#### Назначение:
Отделяет создание сложных объектов от их представления.

#### Пример в Python

```python
class Pizza:
    def __init__(self):
        self.toppings = []

    def add_topping(self, topping):
        self.toppings.append(topping)

    def __str__(self):
        return f"Pizza with {', '.join(self.toppings)}"

class PizzaBuilder:
    def __init__(self):
        self.pizza = Pizza()

    def add_cheese(self):
        self.pizza.add_topping("cheese")
        return self

    def add_pepperoni(self):
        self.pizza.add_topping("pepperoni")
        return self

    def add_olives(self):
        self.pizza.add_topping("olives")
        return self

    def build(self):
        return self.pizza

# Использование
builder = PizzaBuilder()
pizza = builder.add_cheese().add_pepperoni().build()
print(pizza)  # Pizza with cheese, pepperoni
```

#### Когда использовать:

1. Когда объект имеет множество параметров.
2. Когда объект сложен в создании, и важно сохранить читаемость кода.

#### Особенности в Python:
- Python позволяет передавать параметры конструкторам напрямую, что часто упрощает создание объектов без Builder.

### 1. Adapter (Адаптер)

#### Назначение:
Приводит интерфейс одного класса к интерфейсу, ожидаемому клиентом.

#### Пример в Python

```python
class OldPrinter:
    def old_print(self, message):
        print(f"Old printer: {message}")

class PrinterAdapter:
    def __init__(self, old_printer):
        self.old_printer = old_printer

    def print(self, message):
        self.old_printer.old_print(message)

# Использование
old_printer = OldPrinter()
adapter = PrinterAdapter(old_printer)
adapter.print("Hello!")  # Old printer: Hello!
```

#### Когда использовать:

1. Когда необходимо использовать объект с несовместимым интерфейсом.
2. При миграции старого кода.

### 2. Decorator (Декоратор)

#### Назначение:
Динамически добавляет функциональность объекту.

#### Реализация через класс

```python
class Text:
    def render(self):
        return "Hello"

class BoldDecorator:
    def __init__(self, text):
        self.text = text

    def render(self):
        return f"<b>{self.text.render()}</b>"

# Использование
text = Text()
decorated = BoldDecorator(text)
print(decorated.render())  # <b>Hello</b>
```

#### Когда использовать:

1. Для расширения поведения без изменения базового класса.
2. Когда требуется гибкость в добавлении функций.

#### Особенности в Python:
- В Python часто используют функции-декораторы вместо классов.


### 3. Proxy (Заместитель)

#### Назначение

**Proxy (Заместитель)** — это паттерн, который предоставляет объект-заместитель для контроля доступа к реальному объекту. Заместитель может:
1. Контролировать доступ (например, проверять права).
2. Логировать обращения к объекту.
3. Управлять производительностью (например, ленивую инициализацию).

#### Ленивая инициализация

```python
class HeavyService:
    def __init__(self):
        print("Initializing heavy service...")
    
    def operation(self):
        print("Performing heavy operation!")

class ProxyService:
    def __init__(self):
        self._real_service = None

    def operation(self):
        if self._real_service is None:
            self._real_service = HeavyService()  # Инициализация только при первом вызове
        self._real_service.operation()

# Использование
proxy = ProxyService()
proxy.operation()  # Initializing heavy service...
proxy.operation()  # Performing heavy operation!
```

#### Когда использовать Proxy?

1. Для управления доступом к ресурсам (например, через прокси-серверы).
2. Для оптимизации (ленивая инициализация).
3. Для логирования и отладки.

### 1. Observer (Наблюдатель)

#### Назначение

**Observer (Наблюдатель)** — это поведенческий паттерн, который позволяет объектам подписываться на события другого объекта и автоматически уведомляться об изменениях.

#### Реализация

```python
class Subject:
    def __init__(self):
        self._observers = []

    def attach(self, observer):
        self._observers.append(observer)

    def detach(self, observer):
        self._observers.remove(observer)

    def notify(self):
        for observer in self._observers:
            observer.update()

class Observer:
    def update(self):
        raise NotImplementedError("Subclasses must implement 'update'")

class ConcreteObserver(Observer):
    def update(self):
        print("Observer notified!")

# Использование
subject = Subject()
observer1 = ConcreteObserver()
observer2 = ConcreteObserver()

subject.attach(observer1)
subject.attach(observer2)

subject.notify()
# Output:
# Observer notified!
# Observer notified!
```

#### Усовершенствование с передачей данных

```python
class Subject:
    def __init__(self):
        self._observers = []

    def attach(self, observer):
        self._observers.append(observer)

    def notify(self, data):
        for observer in self._observers:
            observer.update(data)

class Observer:
    def update(self, data):
        raise NotImplementedError("Subclasses must implement 'update'")

class ConcreteObserver(Observer):
    def update(self, data):
        print(f"Observer received data: {data}")

# Использование
subject = Subject()
observer = ConcreteObserver()

subject.attach(observer)
subject.notify({"key": "value"})  # Observer received data: {'key': 'value'}
```

#### Когда использовать Observer?

1. Когда необходимо уведомлять множество объектов о событии.
2. Когда объекты должны быть независимы друг от друга, но связаны через событие.


### 2. Strategy (Стратегия)


#### Назначение

**Strategy (Стратегия)** — это поведенческий паттерн, который позволяет определять семейство алгоритмов и делать их взаимозаменяемыми.

#### Пример реализации в Python

```python
class Strategy:
    def execute(self):
        raise NotImplementedError("Subclasses must implement 'execute'")

class ConcreteStrategyA(Strategy):
    def execute(self):
        print("Executing strategy A")

class ConcreteStrategyB(Strategy):
    def execute(self):
        print("Executing strategy B")

class Context:
    def __init__(self, strategy: Strategy):
        self._strategy = strategy

    def set_strategy(self, strategy: Strategy):
        self._strategy = strategy

    def execute_strategy(self):
        self._strategy.execute()

# Использование
context = Context(ConcreteStrategyA())
context.execute_strategy()  # Executing strategy A

context.set_strategy(ConcreteStrategyB())
context.execute_strategy()  # Executing strategy B
```

#### Когда использовать Strategy?

1. Когда требуется выбирать алгоритм выполнения во время выполнения программы.
2. Когда алгоритмы похожи по структуре, но отличаются деталями.

### 3. Command (Команда)


#### Назначение

**Command (Команда)** — это поведенческий паттерн, который превращает запрос в объект, позволяя параметризовать объекты запросами, ставить запросы в очередь или логировать их.

#### Пример реализации в Python

```python
class Command:
    def execute(self):
        raise NotImplementedError("Subclasses must implement 'execute'")

class LightOnCommand(Command):
    def __init__(self, light):
        self.light = light

    def execute(self):
        self.light.turn_on()

class LightOffCommand(Command):
    def __init__(self, light):
        self.light = light

    def execute(self):
        self.light.turn_off()

class Light:
    def turn_on(self):
        print("The light is ON")

    def turn_off(self):
        print("The light is OFF")

class RemoteControl:
    def __init__(self):
        self._commands = []

    def add_command(self, command):
        self._commands.append(command)

    def execute_commands(self):
        for command in self._commands:
            command.execute()

# Использование
light = Light()
remote = RemoteControl()

remote.add_command(LightOnCommand(light))
remote.add_command(LightOffCommand(light))

remote.execute_commands()
# Output:
# The light is ON
# The light is OFF
```

#### Когда использовать Command?

1. Для реализации отмены или повторения действий.
2. Для упрощения взаимодействия между объектами, превращая запросы в объекты.


## Решение задач

### Задача 1: Управление подключениями к базе данных (Singleton)


#### Исходный код:
```python
import sqlite3

class Database:
    def __init__(self, db_name):
        self.connection = sqlite3.connect(db_name)

    def execute_query(self, query):
        cursor = self.connection.cursor()
        cursor.execute(query)
        self.connection.commit()
        return cursor.fetchall()

db1 = Database("app.db")
db2 = Database("app.db")

db1.execute_query("CREATE TABLE IF NOT EXISTS users (id INTEGER PRIMARY KEY, name TEXT)")
db2.execute_query("INSERT INTO users (name) VALUES ('Alice')")

print(db1.execute_query("SELECT * FROM users"))  # [(1, 'Alice')]
print(db2.execute_query("SELECT * FROM users"))  # [(1, 'Alice')]
```

#### Проблемы:
1. Создаются несколько подключений к одной базе данных, что приводит к избыточной нагрузке на ресурсы.
2. Неуправляемая передача объектов приводит к потенциальным проблемам с состоянием.

#### Исправленный код:
```python
import sqlite3

class Database:
    _instance = None

    def __new__(cls, db_name):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
            cls._instance.connection = sqlite3.connect(db_name)
        return cls._instance

    def execute_query(self, query):
        cursor = self.connection.cursor()
        cursor.execute(query)
        self.connection.commit()
        return cursor.fetchall()

db1 = Database("app.db")
db2 = Database("app.db")

db1.execute_query("CREATE TABLE IF NOT EXISTS users (id INTEGER PRIMARY KEY, name TEXT)")
db2.execute_query("INSERT INTO users (name) VALUES ('Alice')")

print(db1.execute_query("SELECT * FROM users"))  # [(1, 'Alice')]
print(db2.execute_query("SELECT * FROM users"))  # [(1, 'Alice')]
```

#### Обоснование:
- Singleton гарантирует, что создаётся только одно подключение к базе данных.
- Все операции используют одно и то же соединение, что предотвращает конфликты состояния.


### Задача 2: Кэширование данных API (Proxy)

#### Исходный код:
```python
import requests

def fetch_data(url):
    response = requests.get(url)
    return response.json()

print(fetch_data("https://jsonplaceholder.typicode.com/posts/1"))
print(fetch_data("https://jsonplaceholder.typicode.com/posts/1"))
```

#### Проблемы:
1. Каждый вызов API делает новый запрос, что приводит к избыточным затратам на сеть.
2. Нет механизма кэширования данных.

#### Исправленный код:
```python
import requests

class APIProxy:
    def __init__(self):
        self._cache = {}

    def fetch_data(self, url):
        if url not in self._cache:
            response = requests.get(url)
            self._cache[url] = response.json()
        return self._cache[url]

proxy = APIProxy()

print(proxy.fetch_data("https://jsonplaceholder.typicode.com/posts/1"))
print(proxy.fetch_data("https://jsonplaceholder.typicode.com/posts/1"))  # Данные из кэша
```

#### Обоснование:
- Proxy добавляет кэширование, чтобы избежать повторных вызовов API.
- Улучшается производительность и сокращается нагрузка на сеть.


### Задача 3: Генерация отчётов разного типа (Factory)

#### Исходный код:
```python
class Report:
    def generate_pdf(self):
        print("PDF Report Generated")

    def generate_csv(self):
        print("CSV Report Generated")

report = Report()
report_type = input("Введите тип отчёта (pdf/csv): ")

if report_type == "pdf":
    report.generate_pdf()
elif report_type == "csv":
    report.generate_csv()
else:
    raise ValueError("Неизвестный тип отчёта")
```

#### Проблемы:
1. Код не масштабируется для добавления новых типов отчётов.
2. Логика выбора типа отчёта размыта и трудна для тестирования.

#### Исправленный код:
```python
class Report:
    def generate(self):
        pass

class PDFReport(Report):
    def generate(self):
        print("PDF Report Generated")

class CSVReport(Report):
    def generate(self):
        print("CSV Report Generated")

class ReportFactory:
    @staticmethod
    def create_report(report_type):
        if report_type == "pdf":
            return PDFReport()
        elif report_type == "csv":
            return CSVReport()
        else:
            raise ValueError("Неизвестный тип отчёта")

report_type = input("Введите тип отчёта (pdf/csv): ")
report = ReportFactory.create_report(report_type)
report.generate()
```

#### Обоснование:
- Factory централизует логику создания объектов.
- Новые типы отчётов добавляются без изменения основной программы.

### Задача 4: Уведомления о состоянии заказа (Observer)

#### Исходный код:
```python
class Order:
    def __init__(self):
        self.status = None

    def set_status(self, status):
        self.status = status
        print(f"Order status updated to: {status}")

order = Order()
order.set_status("Processing")
order.set_status("Shipped")
```

#### Проблемы:
1. Невозможно уведомить сторонние системы или интерфейсы о изменении статуса.
2. Логика уведомлений жёстко встроена в класс `Order`.

#### Исправленный код:
```python
class Order:
    def __init__(self):
        self.status = None
        self._observers = []

    def attach(self, observer):
        self._observers.append(observer)

    def notify(self):
        for observer in self._observers:
            observer.update(self.status)

    def set_status(self, status):
        self.status = status
        print(f"Order status updated to: {status}")
        self.notify()

class EmailNotifier:
    def update(self, status):
        print(f"Sending email: Order status changed to {status}")

class SMSNotifier:
    def update(self, status):
        print(f"Sending SMS: Order status changed to {status}")

order = Order()
order.attach(EmailNotifier())
order.attach(SMSNotifier())

order.set_status("Processing")
order.set_status("Shipped")
```

#### Обоснование:
- Observer уведомляет несколько систем об изменении состояния.
- Логика уведомлений вынесена в отдельные классы, что улучшает читаемость и расширяемость.

### Задача 5: Выбор алгоритма обработки данных (Strategy)

#### Исходный код:
```python
data = [1, 2, 3, 4, 5]
operation = input("Выберите операцию (sum/avg): ")

if operation == "sum":
    print(sum(data))
elif operation == "avg":
    print(sum(data) / len(data))
else:
    raise ValueError("Неизвестная операция")
```

#### Проблемы:
1. Код трудно расширить для новых операций.
2. Логика расчёта жёстко привязана к условиям.

#### Исправленный код:
```python
class Strategy:
    def execute(self, data):
        pass

class SumStrategy(Strategy):
    def execute(self, data):
        return sum(data)

class AvgStrategy(Strategy):
    def execute(self, data):
        return sum(data) / len(data)

class Context:
    def __init__(self, strategy: Strategy):
        self._strategy = strategy

    def set_strategy(self, strategy: Strategy):
        self._strategy = strategy

    def execute(self, data):
        return self._strategy.execute(data)

data = [1, 2, 3, 4, 5]
operation = input("Выберите операцию (sum/avg): ")

strategies = {
    "sum": SumStrategy(),
    "avg": AvgStrategy()
}

if operation in strategies:
    context = Context(strategies[operation])
    print(context.execute(data))
else:
    print("Неизвестная операция")
```

#### Обоснование:
- Strategy позволяет легко добавлять новые алгоритмы.
- Код становится проще для тестирования и масштабирования.


### Задача 6: Управление загрузкой и обработкой изображений (Adapter)

#### Исходный код:
```python
class OldImageProcessor:
    def process(self, image_path):
        print(f"Processing image: {image_path}")

def process_image(image_path, processor):
    processor.process(image_path)

processor = OldImageProcessor()
process_image("image.jpg", processor)
```

#### Проблемы:
1. Если вы хотите перейти на новую библиотеку обработки изображений (например, PIL или OpenCV), придётся переписать всю логику.
2. Нет унифицированного интерфейса для разных процессоров изображений.

#### Исправленный код:
```python
class OldImageProcessor:
    def process(self, image_path):
        print(f"Processing image (old processor): {image_path}")

class NewImageProcessor:
    def apply_filter(self, image_path):
        print(f"Applying filter to image (new processor): {image_path}")

class ImageProcessorAdapter:
    def __init__(self, new_processor):
        self.new_processor = new_processor

    def process(self, image_path):
        self.new_processor.apply_filter(image_path)

def process_image(image_path, processor):
    processor.process(image_path)

old_processor = OldImageProcessor()
new_processor = ImageProcessorAdapter(NewImageProcessor())

process_image("image.jpg", old_processor)
process_image("image.jpg", new_processor)
```

#### Обоснование:
- Adapter позволяет использовать новый процессор без изменения старой логики.
- Упрощается переход на новую библиотеку.
